# Réduction de la dimensionnalité

Dans le calepin précédent qui traitait de la vectorisation de mots, nous avons pris l’exemple d’un mot représenté par une centaine de composantes après entraînement d’un modèle CBOW (*Continuous Bag of words*) sur le roman *Salammbô*. Du point de vue mathématique, il avait pris la forme d’un vecteur d’une centaine d’attributs :

$$
\vec{\text{cité}} = \begin{pmatrix}
    0.0245   \\
    0.0172   \\
    \text{…} \\
    0.0072
\end{pmatrix}
$$

Si l’on voulait représenter cet objet différemment, sous forme tabulaire par exemple, on aboutirait sans doute à un tableau du genre :

|Mot|D1|D2|…|D100|
|-|-|-|-|-|
|Cité|0.0245|0.0172|…|0.0072|

Et si maintenant nous souhaitions le projeter dans un plan en deux dimensions, comment faire ? L’esprit humain n’est pas habitué à se représenter des figures géométriques au-delà de trois dimensions – à part peut-être pour l’octachore, un 4-cube, popularisé sous la dénomination de tesseract, aussi est-il souvent nécessaire d’adopter une stratégie pour réduire leur dimensionnalité. En changeant de perspective, un cube peut se réduire à un carré, un carré à une ligne, une ligne à un point.

![Hypercube 0D to 4D](./images/hypercube_0D-4D.gif)

Vitaly Ostrosablin. – *From Point to Tesseract*. – CC BY-SA 3.0 &lt;https://creativecommons.org/licenses/by-sa/3.0&gt;, via Wikimedia Commons

## Le fléau de la dimension

Dans l’exemple introductif, nos données étaient représentées par une matrice de dimension $1 \times 100$, soit bien plus qu’un tesseract – et on a vu le grabuge que ça a fini par causer aux Avengers ! Épargnons-leur des ennuis à venir et optons pour la réduction de nos données à une dimension moindre. D’autres bénéfices bien réels sont attendus :
- une visualisation facilitée des données ;
- la réduction des coûts de calculs ou d’acquisition ;
- l’amélioration de la qualité de modèles d’apprentissage.

Une fois les avantages énumérés, qu’avons-nous réellement dit du problème ? Pourquoi la dimensionnalité serait-elle un fléau ? L’expression *Curse of dimensionality* vient du mathématicien Richard Ernest Bellman pour qualifier entre autres le coût de résolution d’un problème pour une variable supplémentaire (Richard Ernest Bellman, *Adaptive control processes: a guided tour*, Princeton University Press, 1961). Un statisticien américain, Leo Breiman, fournit un exemple de comparaison. Considérons une matrice de dimension $100 \times 1$ avec des valeurs comprises dans l’intervalle $[0,1]$ :

|Observation|Valeur|
|-|-|
|$n_1$|0.1891|
|$n_2$|0.7023|
|$n_3$|0.0452|
|$n_i$|…|
|$n_{100}$|0.9036|

Ces observations occupent une certaine place dans l’espace unidimensionnel et il est assez simple d’en obtenir une visualisation ou d’inférer des résultats. Si maintenant nous envisageons un espace à seulement dix dimensions, il faudrait $10^{20}$ observations pour obtenir une densité équivalente et permettre une analyse statistique comparable.

Autre exemple peut-être un peu plus parlant : la distance moyenne entre deux points pris au hasard dans un carré de 1 cm de côtés (hypercube 2D unité) sera de 0,52 cm alors qu’elle sera de 0,66 cm dans un cube ! Pour aller plus loin, la distance moyenne entre deux points sera de 4 cm dans un hypercube unité à 10 dimensions, de 12 cm dans un hypercube unité à 1000 dimensions etc. En somme, les espaces de grande dimension sont majoritairement constitués de vide.

Un autre fait amusant est que dans un carré unité, il est très improbable qu’un point tiré au hasard soit proche d’un bord. Les chances d’être à moins de 0,001 cm d’un côté sont à peu près de 0,4 % dans un carré de 1 cm. Et dans les espaces de grande dimension, c’est l’inverse ! Autrement dit, plus on multiplie les variables, plus il y a de risques de rencontrer des valeurs extrêmes dans une dimension !

Pour finir de se convaincre, prenons une dizaine d’observations décrites par deux variables :

|Observation|Variable 1|Variable 2|
|-|-|-|
|$n_1$|1,2|6,3|
|$n_2$|4,3|5|
|$n_3$|3,5|4|
|$n_4$|1|4|
|$n_5$|2.3|5|
|$n_6$|4|9.7|
|$n_7$|6.8|5.4|
|$n_8$|8.1|9|
|$n_9$|6|1|
|$n_{10}$|5|5|

Et projetons-les dans un plan :

![Projection 2D](./images/2D-projection.png)

Le cercle en orange délimite une frontière d’un rayon de 2 unités autour de l’observation $n_{10}$ afin de montrer que ses plus proches voisins sont les observations $n_2$, $n_3$ et $n_7$.

Si maintenant nous projetons les mêmes données dans un espace unidimensionnel en considérant que la variable 2 est inutile pour les décrire, nous voyons que les plus proches voisins ne $n_{10}$ sont maintenant $n_2$, $n_6$ et $n_9$ :

![Projection 1D](./images/1D-projection.png)

## Méthodes de réduction

La réduction de dimension repose sur l’hypothèse, très souvent vérfiée, selon laquelle des données dans un espace *n*-dimensionnel sont situées à proximité d’un sous-espace *k*-dimensionnel. Quel que soit l’algorithme de réduction envisagé, il implique nécessairement une perte d’information qu’il faudra minimiser en le paramétrant justement.

### L’approche naïve

Dans l’exemple précédent, nous avons l’air de rien appliqué une première méthode de réduction de la dimensionnalité : l’élimination de variables. Dans un projet, en fonction de l’objet d’étude, il ne sera pas nécessaire de conserver toutes les variables qui décrivent les observations. Si l’on veut étudier la relation entre la taille et le poids d’un individu, il sera sans doute intéressant de connaître son âge, son sexe, son régime alimentaire, son activité physique, mais inutile de retenir la marque de sa voiture, le nombre de voyages qu’il a effectués ou s’il a fréquenté un établissement hospitalier dans les cinq dernières années.

### Analyse en composantes principales

Parmi les algorithmes de réduction de la dimensionnalité, l’analyse en composantes principales occupe sans conteste la première place. Elle est souvent le premier réflexe lorsque l’on est confronté·e à un jeu de données de grande dimension.

Elle consiste à repérer, dans un jeu de données, les composantes qui vont préserver au maximum la variance. Regardons la projection de la distance et de la vitesse de quelques objets stellaires issus de l’article de Wendy Freedman et al. ("Final Results from the Hubble Space Telescope Key Project to Measure the Hubble Constant". *The Astrophysical Journal*, n° 553, p. 47-72, 2001. https://doi.org/10.48550/arXiv.astro-ph/0012376) :

![Relation between distance and velocity of stellar objects](./images/stellar-objects.png)

Pour conserver au mieux la variance, nous allons chercher un axe qui fournisse le plus grand nombre de coordonnées à ces points :

![Relation between distance and velocity of stellar objects](./images/stellar-objects-variance-axis.png)

À ce nouvel axe, nous prendrons ensuite l’axe orthogonal comme composante secondaire qui caractérise au mieux la variance résiduelle :

![Relation between distance and velocity of stellar objects](./images/stellar-objects-axis-pca.png)

Cela revient dans les faits, pour une ACP sur des données bidimensionnelles, à effectuer une rotation de l’hyperplan.